# Classical and relativistic hydrodynamics

This notebook shows how to formulate and numerically solve the equations of classical and special-relativistic hydrodynamics in 1D, assuming a slab geometry.

We will use a custom python-3 library for the solution of balance laws in 1D, which is implemented using [numpy](http://www.numpy.org/). This notebook also uses [matplotlib](https://matplotlib.org/) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) for the visualization and the root finding routines from [scipy](https://www.scipy.org/). All of these packages are easily available through the [anaconda python distribution](https://anaconda.org/anaconda/python).

In [ ]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
from math import fabs, sqrt
import numpy as np
import PyHRSC1D as hrsc
from scipy.optimize import bisect, brentq

In [ ]:
import matplotlib as mpl

%matplotlib inline

## Newtonian hydrodynamics

We start with the Newtonian hydrodynamics

### Basic equations

The classical equations of hydrodynamics describe the conservation of mass, momentum, and energy in a fluid. In one spatial dimension they are

$$
\begin{align}
    & \partial_t \rho + \partial_x (\rho v) = 0, \\
    & \partial_t S + \partial_x (S v + p) = 0, \tag{1}  \\
    & \partial_t E + \partial_x [(E + p) v] = 0;
\end{align}
$$

where $\rho$ is the density, $v$ the velocity, $S = \rho v$ the linear momentum, $p$ the pressure, $E = \rho \epsilon + \frac{1}{2} \rho v^2$ is the total energy density, and $e$ is the specific internal energy.

Note that these equations need to be closed with the choice of an equation of state $p = p(\rho, \epsilon)$. Here we will use the simple ideal-gas equation of state

$$
    p = (\gamma - 1) \rho \epsilon,
$$

where $\gamma$ is the *adiabatic index*.

In the case of smooth flows, when shocks are not present, the last equation in (1) can be replaced by the equation of entropy conservation

$$
\partial_t (s \rho) + \partial_x (s \rho v) = 0, \tag{2}
$$

where $s$ is the specific entropy of the fluid (not to be confused with the linear momentum $S$ above). When shock waves are present in the flow, it is necessary to solve Eq. (1), which remains valid in the integral sense even when shocks are present.


### Conservation form

They are a particular case of a general family of equations called *balance laws*

$$
    \partial_t \mathbf{F}^0(\mathbf{u}) +
        \partial_x \mathbf{F}^x(\mathbf{u}) =
        \mathbf{S}(\mathbf{u}), \tag{3}
$$

where

$$
    \mathbf{u} = \begin{pmatrix}
        \rho \\
        v \\
        \epsilon
    \end{pmatrix},
    \quad
    \mathbf{F}^0 = \begin{pmatrix}
        \rho \\
        S \\
        E
    \end{pmatrix}\textrm{, and}
    \quad
    \mathbf{F}^x = \begin{pmatrix}
        \rho v \\
        S v  + p \\
        (E + p) v
    \end{pmatrix},
$$

are respectively the *primitive variables*, *conservative variables*, and the *fluxes*. The *source terms* for the Euler equation are simply $\mathbf{S} = 0$.

There is a vast literature on how to solve equations in the form (3) numerically. Here, we are going to use a variant of the so-called [Kurganov-Tadmor central scheme](https://dx.doi.org/10.1006/jcph.2000.6459). The numerical solver implemented in the `PyHRSC1D` library can handle any equation in the form (3), to specialize it to the case of the Euler equations we need to provide the following functions

$$
    \texttt{prim2con}: \mathbf{u} \mapsto \mathbf{F}^0,
    \qquad
    \texttt{con2prim}: \mathbf{F}^0 \mapsto \mathbf{u},
    \qquad
    \texttt{fluxes}: \mathbf{u} \mapsto \mathbf{F}^x.
$$



`PyHRSC1D` will also need an estimate of the propagation velocities of hydrodynamic waves. For the hydrodynamics equations the characteristic waves are the material waves, propagating with velocity $v$, and the sound waves, which propagate with velocity $c_s$ in the frame coming with the fluid ($c_s$ depends on the equation of state). So the characteristic velocities are

$$
    \lambda_\pm = v \pm c_s, \qquad \lambda_c = v.
$$

### Sound waves

#### Linearized equations

The dispersion relation for sound waves can be computed from the dispersion relation of **isentropic** perturbations of Eq. (1) on a constant background $\rho = \mathrm{const}$, $v = 0$. Since we assume an isentropic flow, we can drop the energy conservation equation from Eq (1), since it is equivalent to Eq. (2) which is trivially satisfied. Keeping only the linear terms we find

$$
\begin{align}
    & \partial_t (\delta\rho) + \rho \partial_x (\delta v) = 0, \\
    & \rho \partial_t (\delta v) + \partial_x (\delta p) = 0.
\end{align}
$$

If we derive the first equation in time and the second in space we have

$$
\begin{align}
    & \partial_t^2 (\delta\rho) + \rho \partial_t (\partial_x \delta v) = 0, \\
    & \rho \partial_x \partial_t (\delta v) + \partial_x^2 (\delta p) = 0,
\end{align}
$$

substituting $\rho \partial_t \partial_x \delta v$ in the first equation using the second we find

$$
    \partial_t^2 (\delta\rho) - \partial_x^2 (\delta p) = 0.
$$

Using the chain rule and assuming isentropic perturbations we can write

$$
    \delta p = \left(\frac{\partial p}{\partial \rho}\right)_s \delta \rho =: c_s^2 \delta \rho \tag{4}
$$

Finally, we find that the density fluctuations propagate according to the wave equation with velocity $c_s$ (note that we are working at linear order, so we can neglect changes in $c_s$ due to the fluctuations):

$$
    \partial_t^2 (\delta\rho) - c_s^2 \partial_x^2 (\delta \rho) = 0
$$

#### Thermodynamical relations

To evaluate Eq. (4), we need to express the derive the expression for the pressure as a function of density for *isentropic* fluctuations.

$$
    \mathrm{d} \epsilon = p \frac{\mathrm{d} \rho}{\rho^2}, \tag{5}
$$

using the equation of state we find

$$
    \epsilon = \frac{p}{(\gamma - 1) \rho}.
$$

Using this relation we can rewrite Eq. (5) as

$$
    \frac{1}{\gamma-1} \mathrm{d}\left( \frac{p}{\rho} \right) = p \frac{\mathrm{d}\rho}{\rho^2}, \tag{6}
$$

which, after some manipulation yields

$$
    \gamma \frac{\mathrm{d} \rho}{\rho} = \frac{\mathrm{d} p}{p},
$$

that implies

$$
    p = K \rho^\gamma,
$$

where $K$ is an integration constant. Since the pressure must also satisfy the equation of state we find that

$$
    K = (\gamma - 1) \rho^{1 - \gamma} \epsilon.
$$

Finally we can write

$$
    c_s^2 = \left( \frac{\partial p}{\partial \rho} \right)_S
          = K \gamma \rho^{\gamma - 1} = (\gamma - 1) \gamma \epsilon
$$

### Implementation

First we define an helper class that evaluates the equation of state

In [ ]:
class IdealGas(object):
    """
    A simple ideal-gas EOS
    """
    def __init__(self, gamma=4./3.):
        self.gamma = gamma
    def press(self, rho, eps):
        """
        Computes the pressure given density and specific internal energy
        """
        return (self.gamma - 1.)*rho*eps
    def energy(self, rho, eps):
        """
        Computes the energy density given density and specific internal energy
        """
        return rho*eps
    def csound(self, rho, eps):
        """
        Computes the sound speed given density and specific internal energy
        """
        assert(np.all(eps >= 0))
        return np.sqrt((self.gamma - 1.)*self.gamma*eps)

`PyHRSC1D` requires that we specify the equations we are solving by specializing a generic conservation law class. We need to provide all of the methods listed below:

In [ ]:
help(hrsc.CLaw)

In [ ]:
class Euler(hrsc.CLaw):
    """
    Class implementing the classical hydrodynamics equations
    
    We use the following definitions
    
    prim : (rho, v, eps)
    cons : (D [= rho], S, E)
    """
    nvars = 3
    varnames = ["rho", "v", "eps"]
    def __init__(self, eos):
        self.eos = eos
    def prim2con(self, prim, cons, argv=None):
        # Some simple aliases
        rho, v, eps = prim
        D, S, E = cons
        # Compute the conservative variables
        D[:] = rho
        S[:] = rho*v
        E[:] = rho*(eps + 0.5*v**2)
    def con2prim(self, cons, prim, argv=None):
        rho, v, eps = prim
        D, S, E = cons
        rho[:] = D
        v[:] = S/D
        eps[:] = (E - 0.5*S*v)/D
    def speeds(self, prim, cons, char, argv=None):
        cs = self.eos.csound(prim[0], prim[2])
        v = prim[1]
        char[0,:] = v - cs
        char[1,:] = v
        char[2,:] = v + cs
    def fluxes(self, prim, cons, flux, argv=None):
        rho, v, eps = prim
        D, S, E = cons
        p = self.eos.press(rho, eps)
        flux[0,:] = rho*v
        flux[1,:] = S*v + p
        flux[2,:] = (E + p)*v
    def sources(self, prim, cons, source, argv=None):
        source[:] = 0.

### Example: Sod's shock tube

A classical hydrodynamics test is to consider two regions with two fluids initially at rest separated by a membrane at $x=0$. At $t=0$ the membrane is removed and the fluids interact. This is a so-called shock tube setup. The particular example we consider was originally proposed by Sod in a <a href="https://doi.org/10.1016/0021-9991(78)90023-2">classical paper</a>. The equation of state is with $\gamma = 1.4$ and the initial conditions are described by a left and right state given by

$$
    (\rho_L, v_L, \epsilon_L) = (1, 0, 2.5), \quad
    (\rho_R, v_R, \epsilon_R) = (0.125, 0, 2)
$$

With `PyHRSC1D` we need to create an initial data class to specify the initial conditions

In [ ]:
class SodInitialData(hrsc.InitDataGeneric):
    def apply(self, grid, prim, argv=None):
        """
        Set up the value of the primitive variables.
        """
        rho, v, eps = prim
        rho[:] = np.where(grid.xc < 0, 1.0, 0.125)
        v[:] = 0.
        eps[:] = np.where(grid.xc < 0, 2.5, 2.0)

We setup a uniform grid between $-0.5$ and $0.5$ with $N=100$ grid points. We adopt static boundary conditions, meaning that the value of the solution in the ghost regions is left unchanged. This is how we setup the solver:

In [ ]:
grid_sod = hrsc.make_uniform_grid(-0.5, 0.5, 100)
eos_sod = IdealGas(gamma=1.4)
hydro = Euler(eos_sod)
ic_sod = SodInitialData()
bc_sod = hrsc.BoundaryStatic()
solver_sod = hrsc.KurganovTadmorSolver(grid_sod, hydro, ic_sod, bc_sod)

To ensure the stability of the numerical integration we must ensure that

$$
    c_\max \Delta t \leq \Delta x
$$

where $c_\max$ is the maximum characteristic speed (the maximum between $|\lambda_\pm|$ and $|\lambda_c|$ defined above). To this aim we will set

$$
    \Delta t = \mathrm{CFL}\ \frac{\Delta x}{c_\max}
$$

where $\mathrm{CFL}$ is set to be

In [ ]:
CFL = 0.9

This is a helper script that will run a solver and save some of the data frames. In a real code, we would write this data to disk, to avoid filling up the memory with stuff, but for a 1D problem we can afford to store a lot of data.

In [ ]:
class NumericalSolution(object):
    """
    Helper class for storing the numerical solution
    """
    def __init__(self):
        self.times  = []
        self.frames = []
    def save(self, time, varnames, prim):
        self.times.append(time)
        self.frames.append({})
        for name, ivar in zip(varnames, range(prim.shape[0])):
            self.frames[-1][name] = prim[ivar].copy()

def run_solver(solver, CFL=0.9, t_start=0., t_end=1., nframes=2):
    """
    Helper function to run simulations
    
    * solver  : a conservation law solver from PyHRSC
    * t_start : initial time of the simulation
    * t_end   : final time of the simulation
    * nframes : number of frames to save
    
    Returns
    
    The numerical solution at specific times
    """
    # This creates the initial data
    solver.initialize()
    
    if nframes > 1:
        dt_save = (t_end - t_start)/(nframes - 1)
    else:
        dt_save = np.inf
    t_save_next = dt_save
    solution = NumericalSolution()
    solution.save(t_start, solver.claw.varnames, solver.prim)
    
    t = t_start
    while t < t_end:
        dt = solver.estimate_timestep(CFL)
        if t + dt > t_save_next:
            dt = t_save_next - t
        elif t + dt > t_end:
            dt = t_end - t
        solver.step(dt)
        t += dt
        if t >= t_save_next:
            solution.save(t, solver.claw.varnames, solver.prim)
            print("Time: {}".format(t))
            t_save_next += dt_save
            
    return solution

In [ ]:
sol_sod = run_solver(solver_sod, CFL=CFL, t_start=0, t_end=0.2, nframes=10)
sol_sod.times = np.array(sol_sod.times)
for frame in sol_sod.frames:
    frame["p"] = eos_sod.press(frame["rho"], frame["eps"])

The next cell visualizes the results as a function of time. The solution consist of a rarefaction wave traveling to the left and two discontinuities traveling to the right. The rightmost is a shock wave, while the second one is called *contact discontinuity*, because it contains a jump in the density, but not in the pressure of the fluid.

In [ ]:
@interact(time=FloatSlider(value=0, min=sol_sod.times[0], max=sol_sod.times[-1],
                           step=sol_sod.times[1] - sol_sod.times[0],
                           continuous_update=True))
def plot_sod(time):
    idx = np.argmin(np.abs(sol_sod.times - time))
    plt.figure(figsize=[10,7])
    plt.title("Time: {:.3f}".format(sol_sod.times[idx]))
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["rho"], ".", label=r"$\rho$", color="blue")
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["v"], ".", label=r"$v$", color="red")
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["p"], ".", label=r"$p$", color="green")
    plt.xlim(-0.5, 0.5)
    plt.ylim(-0.1, 1.1)
    plt.legend()
    plt.show()

The solution to the Riemann problem can be solved analytically. I have included a tabulated solution for the Sod problem in the folder `./data`. The code in the next cell plots it. Note how the contact discontinuity is smeared over many cells by the numerical code. This is unfortunately typical for central schemes like the Kurganov-Tadmor scheme we are using here.

In [ ]:
exact = np.load("./data/newt_sod_t0p2.npz")
plt.figure(figsize=[10,7])
plt.title("Time: 0.2")
plt.plot(exact["x"], exact["rho"], "-", label=r"$\rho$ exact", color="blue")
plt.plot(exact["x"], exact["v"], "-", label=r"$v$ exact", color="red")
plt.plot(exact["x"], exact["p"], "-", label=r"$p$ exact", color="green")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["rho"], ".", label=r"$\rho$ numerical", color="blue")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["v"], ".", label=r"$v$ numerical", color="red")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["p"], ".", label=r"$p$ numerical", color="green")
plt.legend()